In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Pará - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_PA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Pará - Produção de Cimento (t),Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade,Pará - Desemprego,Pará - IDH Renda,Pará - Consumo de Cimento (t)
0,2014,1427.236076,9.370048e+07,4.468711e+06,9.571191,7.308848e+07,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,1009.067756,1232.551825,26.867590,22.874833,80.065651,0.788621,0.763242,10.244256,0.651619,2041.730714
1,2015,1386.312966,9.493439e+07,4.396932e+06,9.582867,7.332896e+07,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,1009.084655,1168.300890,26.935981,22.950086,80.146850,0.763383,0.772629,10.537304,0.652508,1863.549771
2,2016,1292.085356,9.599328e+07,4.320024e+06,9.607059,7.362401e+07,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,1009.021941,1146.435512,26.885491,22.912033,80.211878,0.737811,0.771868,10.587975,0.652191,1705.680514
3,2017,1128.424890,9.681084e+07,4.313294e+06,9.643875,7.397420e+07,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,1009.076536,1076.508642,26.811809,22.889590,80.405874,0.723978,0.770765,10.306831,0.652753,1564.940000
4,2018,977.128293,9.698495e+07,4.325703e+06,9.654974,7.405661e+07,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,1009.120505,992.477805,26.765065,22.940164,80.806733,0.722008,0.769289,10.211085,0.653276,1375.281000
5,2019,897.035649,9.683663e+07,4.333697e+06,9.669719,7.412922e+07,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,1008.970794,975.378638,26.804563,23.043376,81.155536,0.717619,0.767406,10.177874,0.652029,1290.928000
6,2020,879.192610,9.595295e+07,4.340025e+06,9.665505,7.397796e+07,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,1008.804964,1008.758026,26.823565,23.300800,82.318139,0.702099,0.765079,10.115271,0.652122,1444.547000
7,2021,897.361640,9.338284e+07,4.350288e+06,9.606219,7.322784e+07,6.638811e+08,4.421848e+09,1.933031e+10,4.488070e+10,...,1008.722801,1060.463592,26.861751,23.493826,83.046153,0.736433,0.763359,10.019973,0.651043,1529.052000
8,2022,908.295113,9.030478e+07,4.362169e+06,9.535341,7.232375e+07,7.010332e+08,4.533216e+09,1.997888e+10,4.699319e+10,...,1008.690851,1089.707540,26.907040,23.512877,82.940212,0.781949,0.765119,10.039894,0.651592,1592.624000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Pará - Produção de Cimento (t),Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade,Pará - Desemprego,Pará - IDH Renda
0,1427.236076,9.370048e+07,4.468711e+06,9.571191,7.308848e+07,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,4.570163e+10,...,36.496952,1009.067756,1232.551825,26.867590,22.874833,80.065651,0.788621,0.763242,10.244256,0.651619
1,1386.312966,9.493439e+07,4.396932e+06,9.582867,7.332896e+07,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,3.539769e+10,...,36.539513,1009.084655,1168.300890,26.935981,22.950086,80.146850,0.763383,0.772629,10.537304,0.652508
2,1292.085356,9.599328e+07,4.320024e+06,9.607059,7.362401e+07,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,3.407411e+10,...,38.175099,1009.021941,1146.435512,26.885491,22.912033,80.211878,0.737811,0.771868,10.587975,0.652191
3,1128.424890,9.681084e+07,4.313294e+06,9.643875,7.397420e+07,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,3.680756e+10,...,41.723496,1009.076536,1076.508642,26.811809,22.889590,80.405874,0.723978,0.770765,10.306831,0.652753
4,977.128293,9.698495e+07,4.325703e+06,9.654974,7.405661e+07,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,4.727230e+10,...,44.780183,1009.120505,992.477805,26.765065,22.940164,80.806733,0.722008,0.769289,10.211085,0.653276
5,897.035649,9.683663e+07,4.333697e+06,9.669719,7.412922e+07,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,7.227332e+10,...,45.421342,1008.970794,975.378638,26.804563,23.043376,81.155536,0.717619,0.767406,10.177874,0.652029
6,879.192610,9.595295e+07,4.340025e+06,9.665505,7.397796e+07,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,9.809293e+10,...,44.516631,1008.804964,1008.758026,26.823565,23.300800,82.318139,0.702099,0.765079,10.115271,0.652122


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    1863.549771
1    1705.680514
2    1564.940000
3    1375.281000
4    1290.928000
5    1444.547000
6    1529.052000
7    1592.624000
8            NaN
Name: Pará - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pará - Produção de Cimento (t),Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade,Pará - Desemprego,Pará - IDH Renda
0,1.212644,-1.811498,1.945462,-1.365770,-1.576087,1.779974,0.019134,2.003355,-1.880579,0.034168,...,-1.094685,0.222379,1.438460,0.394255,-1.092407,-1.019873,1.803948,-1.883232,-0.625016,-1.461198
1,1.008031,-0.784412,0.664142,-1.049509,-0.956538,0.870373,-0.621387,0.401153,0.111653,-0.753105,...,-1.083112,0.572776,0.748447,1.592259,0.273602,-0.812723,0.822401,1.083980,1.207222,0.210599
2,0.536900,0.096990,-0.708731,-0.394240,-0.196414,-0.328802,-0.753466,-0.407217,1.154682,-0.854233,...,-0.638369,-0.727587,0.513627,0.707830,-0.417158,-0.646826,-0.172122,0.843401,1.524038,-0.386546
3,-0.281391,0.777511,-0.828858,0.602948,0.705780,-0.896969,-0.809829,-0.830898,0.955301,-0.645384,...,0.326499,0.404428,-0.237342,-0.582864,-0.824542,-0.151913,-0.710103,0.494737,-0.233776,0.672080
4,-1.037863,0.922436,-0.607356,0.903595,0.918101,-0.983185,0.071542,-0.963455,0.100493,0.154175,...,1.157662,1.316120,-1.139779,-1.401679,0.093487,0.870742,-0.786715,0.028160,-0.832409,1.655939
5,-1.438321,0.798971,-0.464658,1.302976,1.105157,-0.441390,2.094006,-0.202939,-0.441549,2.064379,...,1.332004,-1.788116,-1.323413,-0.709800,1.967018,1.760593,-0.957410,-0.567047,-1.040059,-0.690874


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    1863.549771
1    1705.680514
2    1564.940000
3    1375.281000
4    1290.928000
5    1444.547000
Name: Pará - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Pará - Produção de Cimento (t),Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade,Pará - Desemprego,Pará - IDH Renda
6,-1.22475,0.058698,-0.322772,1.003953,0.639424,1.16956,2.035192,0.985175,-0.705937,2.048732,...,0.930131,-2.185436,-0.839284,-0.345492,2.275496,2.137412,-1.244706,-1.081938,-1.165614,-0.467963


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    1529.052
Name: Pará - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1, restore_best_weights=True)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4096, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [14]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[377288857, 1346284900, 3183925983, 3440814948, 3291794220, 1367128355, 2876217783, 899124459, 3587067437, 1726242367]


Step: 0 ___________________________________________


2023-08-23 17:12:10.750519: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-23 17:12:10.750912: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Restoring model weights from the end of the best epoch: 563.
Epoch 1063: early stopping
val_loss: 4482.720703125
winner_seed: 377288857


Step: 1 ___________________________________________
Restoring model weights from the end of the best epoch: 445.
Epoch 945: early stopping
val_loss: 438.8800048828125
winner_seed: 1346284900


Step: 2 ___________________________________________
Restoring model weights from the end of the best epoch: 1736.
Epoch 2236: early stopping
val_loss: 44.6524772644043
winner_seed: 3183925983


Step: 3 ___________________________________________
Restoring model weights from the end of the best epoch: 830.
Epoch 1330: early stopping
val_loss: 1570.8759765625


Step: 4 ___________________________________________
Restoring model weights from the end of the best epoch: 374.
Epoch 874: early stopping
val_loss: 0.025571882724761963
winner_seed: 3291794220


Step: 5 ___________________________________________
Restoring model weights from the end of the best epoch: 123

In [15]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 381ms/step - loss: 2412497.2500 - val_loss: 2011229.5000
Epoch 2/5000
1/1 [==============================] - 0s 41ms/step - loss: 2382059.0000 - val_loss: 2109869.0000
Epoch 3/5000
1/1 [==============================] - 0s 45ms/step - loss: 2328324.0000 - val_loss: 1710424.6250
Epoch 4/5000
1/1 [==============================] - 0s 43ms/step - loss: 2144105.2500 - val_loss: 1147129.6250
Epoch 5/5000
1/1 [==============================] - 0s 43ms/step - loss: 2137413.7500 - val_loss: 1107724.3750
Epoch 6/5000
1/1 [==============================] - 0s 41ms/step - loss: 2084879.6250 - val_loss: 1085919.7500
Epoch 7/5000
1/1 [==============================] - 0s 47ms/step - loss: 2060147.0000 - val_loss: 1038816.9375
Epoch 8/5000
1/1 [==============================] - 0s 41ms/step - loss: 2010572.8750 - val_loss: 1053707.7500
Epoch 9/5000
1/1 [==============================] - 0s 34ms/step - loss: 2021205.0000 - val_loss: 1183048.1250


1/1 [==============================] - 0s 36ms/step - loss: 714943.8125 - val_loss: 848476.0000
Epoch 75/5000
1/1 [==============================] - 0s 37ms/step - loss: 485776.8438 - val_loss: 968726.1875
Epoch 76/5000
1/1 [==============================] - 0s 34ms/step - loss: 459259.2500 - val_loss: 962236.2500
Epoch 77/5000
1/1 [==============================] - 0s 35ms/step - loss: 489928.2500 - val_loss: 971326.9375
Epoch 78/5000
1/1 [==============================] - 0s 36ms/step - loss: 458281.0000 - val_loss: 977254.1250
Epoch 79/5000
1/1 [==============================] - 0s 35ms/step - loss: 463963.2188 - val_loss: 968594.0625
Epoch 80/5000
1/1 [==============================] - 0s 37ms/step - loss: 465533.4062 - val_loss: 915325.9375
Epoch 81/5000
1/1 [==============================] - 0s 35ms/step - loss: 425913.0312 - val_loss: 972532.1875
Epoch 82/5000
1/1 [==============================] - 0s 35ms/step - loss: 371615.1250 - val_loss: 872594.6250
Epoch 83/5000
1/1 [=====

1/1 [==============================] - 0s 38ms/step - loss: 79960.8828 - val_loss: 285393.9688
Epoch 149/5000
1/1 [==============================] - 0s 37ms/step - loss: 136964.8281 - val_loss: 245238.5938
Epoch 150/5000
1/1 [==============================] - 0s 45ms/step - loss: 138871.5781 - val_loss: 223812.1562
Epoch 151/5000
1/1 [==============================] - 0s 37ms/step - loss: 117761.6953 - val_loss: 228037.2656
Epoch 152/5000
1/1 [==============================] - 0s 42ms/step - loss: 81292.1016 - val_loss: 219950.5781
Epoch 153/5000
1/1 [==============================] - 0s 43ms/step - loss: 143745.2344 - val_loss: 209304.3438
Epoch 154/5000
1/1 [==============================] - 0s 45ms/step - loss: 132208.5625 - val_loss: 195922.9375
Epoch 155/5000
1/1 [==============================] - 0s 43ms/step - loss: 75931.3828 - val_loss: 191888.8125
Epoch 156/5000
1/1 [==============================] - 0s 37ms/step - loss: 97379.2266 - val_loss: 203327.1719
Epoch 157/5000
1/1 [

Epoch 223/5000
1/1 [==============================] - 0s 35ms/step - loss: 36336.4023 - val_loss: 85546.6094
Epoch 224/5000
1/1 [==============================] - 0s 35ms/step - loss: 32615.4004 - val_loss: 89625.2422
Epoch 225/5000
1/1 [==============================] - 0s 34ms/step - loss: 45708.2383 - val_loss: 78674.9766
Epoch 226/5000
1/1 [==============================] - 0s 36ms/step - loss: 44487.1250 - val_loss: 57662.8867
Epoch 227/5000
1/1 [==============================] - 0s 37ms/step - loss: 43739.5000 - val_loss: 41025.8789
Epoch 228/5000
1/1 [==============================] - 0s 36ms/step - loss: 34690.4727 - val_loss: 46709.0664
Epoch 229/5000
1/1 [==============================] - 0s 34ms/step - loss: 33425.9609 - val_loss: 40091.6523
Epoch 230/5000
1/1 [==============================] - 0s 34ms/step - loss: 29243.3496 - val_loss: 57566.2539
Epoch 231/5000
1/1 [==============================] - 0s 36ms/step - loss: 46370.3398 - val_loss: 79005.6016
Epoch 232/5000
1/1 

1/1 [==============================] - 0s 38ms/step - loss: 6203.3750 - val_loss: 13654.2305
Epoch 299/5000
1/1 [==============================] - 0s 36ms/step - loss: 10995.3252 - val_loss: 17858.4473
Epoch 300/5000
1/1 [==============================] - 0s 37ms/step - loss: 7529.7456 - val_loss: 14844.5010
Epoch 301/5000
1/1 [==============================] - 0s 38ms/step - loss: 6579.2593 - val_loss: 15891.1074
Epoch 302/5000
1/1 [==============================] - 0s 37ms/step - loss: 10615.4404 - val_loss: 12185.0928
Epoch 303/5000
1/1 [==============================] - 0s 39ms/step - loss: 6794.3496 - val_loss: 14568.2422
Epoch 304/5000
1/1 [==============================] - 0s 37ms/step - loss: 10166.5137 - val_loss: 11297.4424
Epoch 305/5000
1/1 [==============================] - 0s 35ms/step - loss: 9481.2686 - val_loss: 9962.5840
Epoch 306/5000
1/1 [==============================] - 0s 45ms/step - loss: 15901.2783 - val_loss: 9177.7432
Epoch 307/5000
1/1 [=====================

...

1/1 [==============================] - 0s 36ms/step - loss: 2590.1072 - val_loss: 375.9348
Epoch 685/5000
1/1 [==============================] - 0s 36ms/step - loss: 2977.0696 - val_loss: 671.6968
Epoch 686/5000
1/1 [==============================] - 0s 35ms/step - loss: 4388.5127 - val_loss: 818.0354
Epoch 687/5000
1/1 [==============================] - 0s 36ms/step - loss: 3633.0303 - val_loss: 771.1604
Epoch 688/5000
1/1 [==============================] - 0s 36ms/step - loss: 4934.8765 - val_loss: 616.8177
Epoch 689/5000
1/1 [==============================] - 0s 35ms/step - loss: 8784.3545 - val_loss: 2890.6899
Epoch 690/5000
1/1 [==============================] - 0s 41ms/step - loss: 3847.6133 - val_loss: 1980.1957
Epoch 691/5000
1/1 [==============================] - 0s 34ms/step - loss: 8317.7529 - val_loss: 2369.5457
Epoch 692/5000
1/1 [==============================] - 0s 35ms/step - loss: 3581.5945 - val_loss: 1799.9573
Epoch 693/5000
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 36ms/step - loss: 2984.2861 - val_loss: 756.7469
Epoch 762/5000
1/1 [==============================] - 0s 35ms/step - loss: 11792.2627 - val_loss: 2634.3884
Epoch 763/5000
1/1 [==============================] - 0s 34ms/step - loss: 3612.4238 - val_loss: 2257.3635
Epoch 764/5000
1/1 [==============================] - 0s 36ms/step - loss: 2909.4397 - val_loss: 1221.5845
Epoch 765/5000
1/1 [==============================] - 0s 34ms/step - loss: 7798.5503 - val_loss: 3580.5439
Epoch 766/5000
1/1 [==============================] - 0s 35ms/step - loss: 5166.6270 - val_loss: 1183.7565
Epoch 767/5000
1/1 [==============================] - 0s 39ms/step - loss: 3040.8972 - val_loss: 937.1431
Epoch 768/5000
1/1 [==============================] - 0s 37ms/step - loss: 10057.6885 - val_loss: 2988.7871
Epoch 769/5000
1/1 [==============================] - 0s 40ms/step - loss: 3172.0745 - val_loss: 1301.3052
Epoch 770/5000
1/1 [==============================] 

1/1 [==============================] - 0s 38ms/step - loss: 15291.8828 - val_loss: 2093.2524
Epoch 839/5000
1/1 [==============================] - 0s 39ms/step - loss: 5556.1504 - val_loss: 1976.9703
Epoch 840/5000
1/1 [==============================] - 0s 36ms/step - loss: 3851.2019 - val_loss: 1064.4385
Epoch 841/5000
1/1 [==============================] - 0s 37ms/step - loss: 3007.9990 - val_loss: 618.2374
Epoch 842/5000
1/1 [==============================] - 0s 36ms/step - loss: 12629.1523 - val_loss: 0.0028
Epoch 843/5000
1/1 [==============================] - 0s 38ms/step - loss: 2672.1440 - val_loss: 311.6699
Epoch 844/5000
1/1 [==============================] - 0s 38ms/step - loss: 3210.5225 - val_loss: 184.4252
Epoch 845/5000
1/1 [==============================] - 0s 36ms/step - loss: 3417.0579 - val_loss: 160.7095
Epoch 846/5000
1/1 [==============================] - 0s 38ms/step - loss: 2964.4326 - val_loss: 198.7336
Epoch 847/5000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 40ms/step - loss: 619.7139 - val_loss: 520.9449
Epoch 916/5000
1/1 [==============================] - 0s 38ms/step - loss: 1638.5275 - val_loss: 352.3228
Epoch 917/5000
1/1 [==============================] - 0s 36ms/step - loss: 3840.9915 - val_loss: 494.1992
Epoch 918/5000
1/1 [==============================] - 0s 38ms/step - loss: 824.2546 - val_loss: 600.5312
Epoch 919/5000
1/1 [==============================] - 0s 37ms/step - loss: 678.4007 - val_loss: 716.1569
Epoch 920/5000
1/1 [==============================] - 0s 37ms/step - loss: 3601.9402 - val_loss: 574.9341
Epoch 921/5000
1/1 [==============================] - 0s 37ms/step - loss: 2781.4949 - val_loss: 348.1969
Epoch 922/5000
1/1 [==============================] - 0s 36ms/step - loss: 1696.9102 - val_loss: 763.9974
Epoch 923/5000
1/1 [==============================] - 0s 39ms/step - loss: 2192.6738 - val_loss: 477.2241
Epoch 924/5000
1/1 [==============================] - 0s 38ms/st

1/1 [==============================] - 0s 40ms/step - loss: 5319.7007 - val_loss: 382.8968
Epoch 993/5000
1/1 [==============================] - 0s 34ms/step - loss: 1570.2391 - val_loss: 594.2834
Epoch 994/5000
1/1 [==============================] - 0s 36ms/step - loss: 1010.9568 - val_loss: 523.4220
Epoch 995/5000
1/1 [==============================] - 0s 38ms/step - loss: 1458.6001 - val_loss: 355.0455
Epoch 996/5000
1/1 [==============================] - 0s 37ms/step - loss: 5328.4351 - val_loss: 1396.9728
Epoch 997/5000
1/1 [==============================] - 0s 38ms/step - loss: 5668.2109 - val_loss: 230.3896
Epoch 998/5000
1/1 [==============================] - 0s 36ms/step - loss: 3119.7800 - val_loss: 747.4404
Epoch 999/5000
1/1 [==============================] - 0s 40ms/step - loss: 2790.9026 - val_loss: 736.6467
Epoch 1000/5000
1/1 [==============================] - 0s 37ms/step - loss: 11948.4756 - val_loss: 1981.4017
Epoch 1001/5000
1/1 [==============================] - 0s

In [16]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[1529.052](test_target) - [[1444.5972]](prediction) = 84.45483203124991


In [17]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [18]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Pará - Produção de Cimento (t),Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade,Pará - Desemprego,Pará - IDH Renda
0,1.037178,-1.254690,1.210423,-1.060973,-1.181106,1.165029,1.396038,1.335266,-1.330895,1.406550,...,-0.734171,0.363799,1.371385,-0.993308,-1.220083,-1.267309,1.222055,-1.410442,-1.400969,-1.323476
1,0.313995,0.062267,0.028158,-0.279299,-0.083050,0.111769,-0.502296,-0.264151,0.251276,-0.575945,...,-0.679693,1.001628,-0.386566,1.368436,1.229354,0.090106,0.005361,0.794613,0.533259,1.093372
2,-1.351173,1.192424,-1.238581,1.340272,1.264156,-1.276798,-0.893743,-1.071116,1.079619,-0.830605,...,1.413864,-1.365427,-0.984819,-0.375128,-0.009270,1.177204,-1.227417,0.615829,0.867710,0.230104


train_target:


,Pará - Consumo de Cimento (t)
0,1863.549771
1,1705.680514
2,1564.940000


test_input:


...

,Pará - Produção de Cimento (t),Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade,Pará - Desemprego,Pará - IDH Renda
6,-1.22475,0.058698,-0.322772,1.003953,0.639424,1.16956,2.035192,0.985175,-0.705937,2.048732,...,0.930131,-2.185436,-0.839284,-0.345492,2.275496,2.137412,-1.244706,-1.081938,-1.165614,-0.467963


test_target:


,Pará - Consumo de Cimento (t)
6,1529.052


Restoring model weights from the end of the best epoch: 533.
Epoch 1033: early stopping
1/1 [==============================] - 0s 38ms/step
Error: 84.45483203124991




[1564.9149169921875, 1375.713623046875, 1291.0628662109375, 1444.59716796875]

In [19]:
display(targets)
display(predictions)

[1375.281, 1290.928, 1444.547, 1529.052]

[1564.9149169921875, 1375.713623046875, 1291.0628662109375, 1444.59716796875]

In [20]:
mae = mean_absolute_error(predictions, targets)
mae

128.08962646484372

In [21]:
porcentage = mae/np.mean(targets)
porcentage

0.09084679936965495